# PyRadiomics Sample Notebook

This notebook demonstrates how to convert Dicom data to Nifti and generate radiomics features using pyradiomics

In [ ]:
import sys, os
sys.path.append('../..')

import SimpleITK as sitk
import pandas as pd
from radiomics import firstorder, glcm

from impit.dicom.rtstruct_to_nifti.convert import convert_rtstruct

### Convert image series
First the image series is converted to Nifti

In [ ]:
# Convert Image series
path_to_image_series = "../dicom/data/phantom/CT"
series_files = sitk.ImageSeriesReader().GetGDCMSeriesFileNames(path_to_image_series)
image = sitk.ReadImage(series_files)

### Convert contours
Next the contours within an RTStruct are converted to individual Nifti masks

In [ ]:
# Convert RTStructs to masks
path_to_rt_struct = "../dicom/data/phantom/RTStruct.dcm"
pre = 'Test_'
output_dir = 'test_output_nifti'
output_img = 'img.nii.gz'

# Run the function
convert_rtstruct(path_to_image_series, path_to_rt_struct, prefix=pre, output_dir=output_dir, output_img=output_img)

### Extract Radiomics
Finally, radiomics are extracted within each contour using pyradiomics

In [ ]:
# pyradiomics settings
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = None
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True

# Run pyradiomics on each of the masks
nifti_image_path = os.path.join(output_dir, output_img)
image = sitk.ReadImage(nifti_image_path)

# For each mask in the nifti directory
rad = {}
for f in os.listdir(output_dir):
    
    if not f.startswith(pre):
        continue
        
    print("Processing: {0}".format(f))
    
    mask_file = os.path.join(output_dir, f)
    mask = sitk.ReadImage(mask_file)

    firstOrderFeatures = firstorder.RadiomicsFirstOrder(image, mask, **settings)
    firstOrderFeatures.enableAllFeatures()
    firstorderresult = firstOrderFeatures.execute()

    glcmFeatures = glcm.RadiomicsGLCM(image, mask, **settings)
    glcmFeatures.enableAllFeatures()
    glcmresult = glcmFeatures.execute()

    # Merge the results:
    result = {**firstorderresult, **glcmresult}

    contour_name = mask_file.split('.')[0].split('\\')[-1]

    rad[contour_name] = result

# Convert to Pandas dataframe
rad_frame = pd.DataFrame.from_dict(rad)
rad_frame = rad_frame.transpose()
display(rad_frame)